Получение датесета

Шутки взяты из датасета https://github.com/Koziev/NLP_Datasets/blob/master/Conversations/Data/extract_dialogues_from_anekdots.tar.xz

Просмотр статистики и объема датасета

Датасет имеет примерно 80к шуток, для обучения выбраны первые 5000. Шутки содержат до 300 символов текста

In [1]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# использовать из низа модель
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Downloading: 100%|██████████| 1.61M/1.61M [00:01<00:00, 1.41MB/s]
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 1.27M/1.27M [00:01<00:00,

In [3]:
#Предобработка данных

In [3]:
from datasets import load_dataset, Dataset

In [4]:
dataset = load_dataset("text", data_files={"train": "./train_dataset.txt"}, sample_by="document")

Using custom data configuration default-e7cb14cc9e70c1c8
Found cached dataset text (C:/Users/USER/.cache/huggingface/datasets/text/default-e7cb14cc9e70c1c8/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)
100%|██████████| 1/1 [00:00<00:00, 142.82it/s]


In [5]:
joke_datasets = []

In [6]:
k = 0
for index, item in enumerate(dataset['train'][0]['text'].split('\n\n\n\n')):
    k = k +1
    if (k > 60): #ВОт тут длинна датасета !!!!!
        break
    joke_datasets.append({"text":item})

In [7]:
dataset = Dataset.from_list(joke_datasets, split="train+test")

In [8]:
dataset[0]

{'text': '[SJ]- Как водичка ?\n- А я здесь как женшина сижу, а не как термометр.[EJ]'}

In [9]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"]))

100%|██████████| 60/60 [00:00<00:00, 1094.80ex/s]


In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Выдвижение гипотез


Модель будет генерировать шутки похожие на те, что в датасете и дополнять их

Выбор модели

Для генерации текста выбрана gpt3: rugpt3small_based_on_gpt2

Обучение модели

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuner",
    learning_rate= 2e-3,
    per_device_train_batch_size = 1, # если позволяет память - можно сделать больше
    per_device_eval_batch_size=1,
    num_train_epochs=200, # сколько будет пройден датасет полностью
    weight_decay=0.01,
    save_total_limit = 1
)


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
        optimizers = (torch.optim.AdamW(model.parameters(),lr=2e-3),None) # Optimizer and lr scheduler
)

In [13]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Assigning [PAD] to the pad_token key of the tokenizer
Adding [PAD] to the vocabulary


1

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 60
  Num Epochs = 200
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 12000
  Number of trainable parameters = 355871744
  4%|▍         | 500/12000 [11:12<4:16:37,  1.34s/it]Saving model checkpoint to ./finetuner\checkpoint-500
Configuration saved in ./finetuner\checkpoint-500\config.json


{'loss': 4.5733, 'learning_rate': 0.0019166666666666668, 'epoch': 8.33}


Model weights saved in ./finetuner\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./finetuner\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./finetuner\checkpoint-500\special_tokens_map.json
added tokens file saved in ./finetuner\checkpoint-500\added_tokens.json
  8%|▊         | 1000/12000 [22:32<4:06:33,  1.34s/it]Saving model checkpoint to ./finetuner\checkpoint-1000
Configuration saved in ./finetuner\checkpoint-1000\config.json


{'loss': 3.4981, 'learning_rate': 0.0018333333333333333, 'epoch': 16.67}


Model weights saved in ./finetuner\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./finetuner\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./finetuner\checkpoint-1000\special_tokens_map.json
added tokens file saved in ./finetuner\checkpoint-1000\added_tokens.json
Deleting older checkpoint [finetuner\checkpoint-500] due to args.save_total_limit
 12%|█▎        | 1500/12000 [33:53<3:57:32,  1.36s/it] Saving model checkpoint to ./finetuner\checkpoint-1500
Configuration saved in ./finetuner\checkpoint-1500\config.json


...

Model weights saved in ./finetuner\checkpoint-11500\pytorch_model.bin
tokenizer config file saved in ./finetuner\checkpoint-11500\tokenizer_config.json
Special tokens file saved in ./finetuner\checkpoint-11500\special_tokens_map.json
added tokens file saved in ./finetuner\checkpoint-11500\added_tokens.json
Deleting older checkpoint [finetuner\checkpoint-11000] due to args.save_total_limit
100%|██████████| 12000/12000 [4:31:20<00:00,  1.33s/it]Saving model checkpoint to ./finetuner\checkpoint-12000
Configuration saved in ./finetuner\checkpoint-12000\config.json


{'loss': 1.4022, 'learning_rate': 0.0, 'epoch': 200.0}


Model weights saved in ./finetuner\checkpoint-12000\pytorch_model.bin
tokenizer config file saved in ./finetuner\checkpoint-12000\tokenizer_config.json
Special tokens file saved in ./finetuner\checkpoint-12000\special_tokens_map.json
added tokens file saved in ./finetuner\checkpoint-12000\added_tokens.json
Deleting older checkpoint [finetuner\checkpoint-11500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 12000/12000 [4:31:27<00:00,  1.36s/it]

{'train_runtime': 16287.8763, 'train_samples_per_second': 0.737, 'train_steps_per_second': 0.737, 'train_loss': 2.425979482014974, 'epoch': 200.0}


TrainOutput(global_step=12000, training_loss=2.425979482014974, metrics={'train_runtime': 16287.8763, 'train_samples_per_second': 0.737, 'train_steps_per_second': 0.737, 'train_loss': 2.425979482014974, 'epoch': 200.0})

Расчет метрик

loss полученный при обучении модели равен 0.4

Сохранение модели. 

Модель сохранена в папку checkpoint-125000, для загрузки используется следующий код:

In [1]:
from transformers import AutoConfig, GPT2Config
label2id= {
    "negative":0,
    "positive":1,
}
id2label= {
    0:"negative",
    1:"positive",
}
config = GPT2Config.from_pretrained("./checkpoint-125000", label2id=label2id, id2label=id2label)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = GPT2Tokenizer.from_pretrained("./checkpoint-125000")
model = GPT2LMHeadModel.from_pretrained("./checkpoint-125000",config=config).to(DEVICE) # Загружаем обученную модель из чекпоинтов

Генерация анегдота

In [20]:
text = "[SJ]"
input_ids = tokenizer.encode(text, return_tensors="pt").to("cuda:0")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100, pad_token_id=tokenizer.eos_token_id,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



[SJ]- Опять водка есть?
- Садятся в лужне, Винне...[EJ]
- А как как как него и храма.[EJ]EJ]EJ]EJ]EJ]EJ]EJ]EJ]EJ]
- ТEJ] Т.[EJ]EJ], а неJ] не было.[E]![EJ]EJ]EJ] не не было


Вывод

В данной ЛР я узнал как подготавливать данные для обучения, а также обучать, и запускать модель.